### prueba scrapear!

In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

#### carrefour

In [100]:
ua = UserAgent()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36'}

list_pages = list(range(0,336,24))
table_pages=[]
for number in list_pages:
    url = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all({'article':'product-card-item'})
    table_pages.append(table)


In [158]:
table_pages[0][24]

<article class="product-card-item">
<div class="load-more-products">
<p>SIGUIENTE PÁGINA</p>
<a class="pathCart" href="/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No=24&amp;Nr%3DAND%28product.shopCodes%3A004320%2Cproduct.salepointWithActivePrice_004320%3A1%2COR%28product.siteId%3AbasicSite%29%29OR%29">
</a>
</div>
</article>

In [102]:
len(table_pages[0])

25

In [163]:
data_raw=[]
for i in range(len(table_pages)):
    for j in range(len(table_pages[i])):
        rows = table_pages[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','title-product','js-gap-product-click-super']})
        if len(rows) != 0:
            data_raw.append(rows)


In [166]:
data_raw[22]

[<a class="js-gap-product-click-super" data-product-discount="0,07" data-productid="762603515" data-special-campaign="qvpromo730001_feliz-aniversario" href="/supermercado/cerveza-1906-reserva-especial-pack-de-12-botellas-33-cl-1906/R-762603515/p" title="Cerveza 1906 reserva especial pack de 12 botellas 33 cl.">
 <img alt="Cerveza 1906 reserva especial pack de 12 botellas 33 cl." aria-describedby="product-762603515" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/771393_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/771393_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/771393_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/771393_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/771393_00_1.jpg  80w" width="100%"/>
 </a>,
 <span class="price-less">
 11,40 €</span>,
 <p class="

In [144]:
len(data_raw)

333

In [174]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return re.sub(",",'.',price)

In [180]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return price_liter

In [181]:
#title
# title_raw = data_raw[0][3].text
def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [182]:
data_beer_carrefour = pd.DataFrame(index=range(1,len(data_raw)+1),columns=['price','price_liter','title'])
for beer_number in range(len(data_raw)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw[beer_number][3])



In [183]:
data_beer_carrefour.head(30)

,price,price_liter,title
1,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.
2,14.80,1.60,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...
3,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.
4,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.
5,11.70,1.95,Cerveza Estrella Galicia especial pack de 24 b...
6,14.15,1.53,Cerveza Mahou Clásica pack de 28 latas de 33 cl.
7,15.69,1.70,Cerveza Mahou 5 Estrellas especial pack de 28 ...
8,6.47,1.63,Cerveza Mahou Clásica pack de 12 latas de 33 cl.
9,6.41,1.94,Cerveza Estrella Galicia especial pack de 10 l...
10,6.75,1.70,"Cerveza Mahou 0,0 sin alcohol tostada pack de ..."
